In [3]:
import numpy as np
import scipy.stats as st

RISK_FREE = 0.043  # 4.3% annual

def portfolio_return(weights, mu):
    """Annualised expected return."""
    return np.dot(weights, mu)

def portfolio_vol(weights, cov):
    """Annualised volatility (std dev)."""
    return np.sqrt(weights @ cov @ weights)

def sharpe_ratio(weights, mu, cov):
    """Sharpe ratio: (E[R] - Rf) / σ."""
    ret = portfolio_return(weights, mu)
    vol = portfolio_vol(weights, cov)
    return (ret - RISK_FREE) / vol

def cvar_calc(weights, mu, cov, alpha=0.95):
    """
    Approximate CVaR under Normal assumption:
      CVaR = -(mean - σ * φ(Ζα)/(1-α))
    where Ζα = norm.ppf(α)
    """
    ret = portfolio_return(weights, mu)
    vol = portfolio_vol(weights, cov)
    z = st.norm.ppf(alpha)
    pdf = st.norm.pdf(z)
    cvar = -(ret - vol * pdf / (1 - alpha))
    return cvar


In [4]:
from cvxopt import matrix, solvers

def mean_variance_opt(mu, cov, target_return):
    """
    Solve:
      minimize 0.5 w^T Σ w
      s.t.     μ^T w >= target_return
               sum(w) = 1
               w >= 0
    Returns optimal weights as a NumPy array.
    """
    n = len(mu)
    P = matrix(cov * 2)                 # 2Σ
    q = matrix(np.zeros(n))             # zero vector

    # Constraints Gx <= h  <=>  -Ix <= 0  (w >= 0)
    G = matrix(-np.eye(n))
    h = matrix(np.zeros(n))

    # Constraints Ax = b for equalities and inequalities combined
    # We'll stack [1^T; μ^T] for equalities, then the inequality μ^T w >= target => -μ^T w <= -target
    A = matrix(np.vstack([np.ones((1, n)), mu.reshape(1, n)]))
    b = matrix([1.0, target_return])

    # To handle μ^T w >= target, we include it as an inequality by appending to G,h:
    # -μ^T w <= -target → G = [ -I; -μ ]  and  h = [ 0; -target ]
    G = matrix(np.vstack([np.eye(n)*-1, mu.reshape(1,n)*-1]))
    h = matrix(np.hstack([np.zeros(n), -target_return]))

    sol = solvers.qp(P, q, G, h, A, b)
    w = np.array(sol['x']).flatten()
    return w

In [48]:
import os, json, requests, time, itertools, math
import pandas as pd, numpy as np

API_KEY = os.getenv("ALPHA_VANTAGE_KEY")
BASE    = "https://www.alphavantage.co/query"

# 10-ticker “mini-universe” (7 blue-chip stocks + 3 broad ETFs)
UNIVERSE = [
    # 7 Stocks
    "AMZN","GOOG",
    "BRK.B","JPM",
    "MA","XOM",
    "CVX",

    # 3 ETFs
    "SPY", "VTI","AGG"
]


 ## Fetch weekly‐adjusted prices (cache locally)

In [42]:
from pathlib import Path
CACHE = Path("av_cache"); CACHE.mkdir(exist_ok=True)

def av_get(function, symbol):
    key = f"{function}_{symbol}.json"
    fp = CACHE / key
    if fp.exists():
        return json.loads(fp.read_text())
    url = f"{BASE}?function={function}&symbol={symbol}&apikey={API_KEY}"
    r = requests.get(url); r.raise_for_status()
    fp.write_text(r.text)
    time.sleep(12)                       # AV free tier = 5 calls/min
    return r.json()

def weekly_series(sym):
    js = av_get("TIME_SERIES_WEEKLY_ADJUSTED", sym)
    df = (pd.DataFrame(js["Weekly Adjusted Time Series"])
            .T.astype(float)[["4. close", "7. dividend amount"]].rename(columns={"4. close":"close", "7. dividend amount": "div"}))
    return df.sort_index()

def ttm_div_yield_from_weekly(sym, weeks=52):
    """
    12-month dividend yield using WEEKLY adjusted data.
    - Sums the last `weeks` of dividend amounts (cash).
    - Divides by the latest close.
    - Returns 0.0 for non-payers or missing data.
    """
    df = weekly_series(sym)
    if df.empty or "close" not in df:
        return 0.0

    last_close = float(df["close"].iloc[-1])
    if not np.isfinite(last_close) or last_close <= 0:
        return 0.0

    div_sum = float(df.get("div", 0.0).tail(min(weeks, len(df))).sum())
    if not np.isfinite(div_sum) or div_sum < 0:
        return 0.0

    return float(div_sum / last_close)  # e.g., 0.018 = 1.8%
    
def overview(sym):
    return av_get("OVERVIEW", sym)


## Unified Price Table and Momentum 

In [7]:
price_dfs = {}
for sym in UNIVERSE:
    try:
        price_dfs[sym] = weekly_series(sym)["close"]
    except Exception as e:
        print("skip", sym, e)

prices = pd.concat(price_dfs, axis=1).dropna(how="all")
rets   = prices.pct_change().dropna()
μ      = rets.mean() * 52                        # annualised mean
Σ      = rets.cov()  * 52                        # annualised cov

# Momentum 6m / 12m
mom_6  = prices.pct_change(26).iloc[-1]
mom_12 = prices.pct_change(52).iloc[-1]


/var/folders/7w/gyry33gn15d83nx1ntkgpqg80000gn/T/ipykernel_3253/1920037014.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets   = prices.pct_change().dropna()
/var/folders/7w/gyry33gn15d83nx1ntkgpqg80000gn/T/ipykernel_3253/1920037014.py:14: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  mom_6  = prices.pct_change(26).iloc[-1]
/var/folders/7w/gyry33gn15d83nx1ntkgpqg80000gn/T/ipykernel_3253/1920037014.py:15: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling

In [21]:
price_dfs

{'AMZN': 1999-11-12     74.94
 1999-11-19     77.94
 1999-11-26     93.13
 1999-12-03     86.56
 1999-12-10    106.70
                ...  
 2025-07-25    231.44
 2025-08-01    214.75
 2025-08-08    222.69
 2025-08-15    231.03
 2025-08-19    228.01
 Name: close, Length: 1346, dtype: float64,
 'GOOG': 2014-04-04    543.14
 2014-04-11    530.60
 2014-04-17    536.10
 2014-04-25    516.18
 2014-05-02    527.93
                ...  
 2025-07-25    194.08
 2025-08-01    189.95
 2025-08-08    202.09
 2025-08-15    204.91
 2025-08-21    200.62
 Name: close, Length: 595, dtype: float64,
 'BRK.B': 1999-11-12    1968.00
 1999-11-19    1855.00
 1999-11-26    1883.00
 1999-12-03    1840.00
 1999-12-10    1787.00
                ...   
 2025-07-25     484.07
 2025-08-01     472.84
 2025-08-08     465.40
 2025-08-15     477.20
 2025-08-21     488.59
 Name: close, Length: 1346, dtype: float64,
 'JPM': 1999-11-12     85.00
 1999-11-19     83.00
 1999-11-26     79.06
 1999-12-03     82.00
 1999-12-10 

## Fundamental Data 

In [43]:
def etf_profile(sym):
    """
    Try Alpha Vantage ETF profile (cached via av_get just like other calls).
    If unavailable, return {}. Endpoint name is 'ETF_PROFILE'.
    """
    return av_get("ETF_PROFILE", sym)
    

def last_close_from_weekly(sym):
    df = weekly_series(sym)
    if df.empty or "close" not in df:
        return np.nan
    try:
        return float(df["close"].iloc[-1])
    except Exception:
        return np.nan

def is_etf(sym):
    print(bool(etf_profile(sym)))
    return bool(etf_profile(sym))
    
   

def compute_beta_from_prices(sym, prices, market = "SPY"):
    """
    Estimate beta from weekly returns vs a market proxy (default SPY) using OLS slope.
    Returns np.nan if not enough data.
    """
    if sym not in prices or market not in prices:
        return np.nan
    rets = prices[[sym, market]].pct_change().dropna()
    if len(rets) < 12:
        return np.nan
    x = rets[market].values
    y = rets[sym].values
    denom = np.dot(x, x)
    if denom <= 0:
        return np.nan
    return float(np.dot(x, y) / denom)

def _to_float(x):
    try:
        if x in (None, "None", "", "NaN"):
            return np.nan
        return float(x)
    except Exception:
        return np.nan

    
fund_rows = []
for sym in UNIVERSE:
    last_close = last_close_from_weekly(sym)
    divy_ttm = ttm_div_yield_from_weekly(sym)
    
    if is_etf(sym):
        # ETF branch: sector fixed, shares outstanding from ETF_PROFILE (if present)
        prof = etf_profile(sym) 
        shares_out = _to_float(prof.get("SharesOutstanding"))
        # Market cap = shares * price (if both available)
        mcap = shares_out * last_close if np.isfinite(shares_out) and np.isfinite(last_close) else np.nan
        logcap = math.log(mcap) if np.isfinite(mcap) and mcap > 0 else np.nan
        # ETF beta: try OVERVIEW beta if present (some AV ETFs include it), else estimate from prices vs SPY
        beta = compute_beta_from_prices(sym, prices, market="SPY")    
        sector = "ETF"
    else:
        o = overview(sym)
        beta     = _to_float(o.get("Beta"))
        mcap     = _to_float(o.get("MarketCapitalization"))
        logcap   = math.log(mcap) if np.isfinite(mcap) and mcap > 0 else np.nan
        sector   = o.get("Sector") or "N/A"

     # 1) ALWAYS append a row (no try/except swallow)
    fund_rows.append({
        "ticker":   sym,
        "beta":     beta,
        "divYield": divy_ttm,
        "logCap":   logcap,
        "sector":   sector,
    })


fund = pd.DataFrame(fund_rows).set_index("ticker")



False
True


/var/folders/7w/gyry33gn15d83nx1ntkgpqg80000gn/T/ipykernel_3253/3128305470.py:31: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices[[sym, market]].pct_change().dropna()


KeyError: 'Weekly Adjusted Time Series'

In [44]:
av_get("TIME_SERIES_WEEKLY_ADJUSTED", 'BRK-B')

{'Information': 'We have detected your API key as None and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [73]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=&symbol=BRK.B&apikey=P8DFNFHFJTNUZ8CP'
r = requests.get(url)
data = r.json()

print(data)

{'Information': 'We have detected your API key as P8DFNFHFJTNUZ8CP and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


In [51]:
fund

,beta,divYield,logCap,sector
ticker,,,,
AMZN,1.314,0.000000,28.519613,TRADE & SERVICES
GOOG,1.014,0.004037,28.524157,TECHNOLOGY
JPM,1.107,0.018184,27.406930,FINANCE
MA,1.029,0.004967,26.994837,TRADE & SERVICES
XOM,0.502,0.036254,26.850044,ENERGY & TRANSPORTATION
CVX,0.841,0.043459,26.447676,ENERGY & TRANSPORTATION


## Building Training Data via Simulations

In [57]:
from sklearn.model_selection import train_test_split

def simulate_portfolios(num=2000, size=None, k_frac = (0.2, 0.6), rng_seed = 0):
    rng = np.random.default_rng(rng_seed)
    universe = list(dict.fromkeys(UNIVERSE))
    N = len(universe) 
    
    if size is None:
        k_min = max(3, int(np.ceil(k_frac[0] * N)))
        k_max = max(k_min, min(N, int(np.floor(k_frac[1] * N))))
        ks = rng.integers(k_min, k_max + 1, size=num)
    else:
        k_fixed = min(int(size), N)
        ks = np.full(num, k_fixed, dtype=int)
                     
    sims = []
    for k in ks:
        basket = rng.choice(universe, size=k, replace=False)
        w0     = rng.dirichlet(np.ones(k))
        sims.append(dict(zip(basket, w0)))
    return sims

def label_asset(port, asset, target, tol=1e-4):
    """
    Returns 1 if, at the given target return, adding `asset` and re-optimizing
    the *basket+asset* portfolio raises Sharpe above the current basket Sharpe
    by more than `tol`. Else 0.
    """
    # build the current basket + candidate
    basket  = list(port.keys()) + [asset]
    w0      = np.append(list(port.values()), 0.0)  # current basket, 0 weight for new asset

    # slice μ and Σ to the combined basket (convert to numpy arrays)
    mu_b    = μ[basket].values
    cov_b   = Σ.loc[basket, basket].values
    # small ridge to avoid singular KKT / make SPD
    cov_b   = cov_b + 1e-6 * np.eye(cov_b.shape[0])

    # baseline Sharpe for the existing basket only (exclude the last asset)
    base = sharpe_ratio(
        w0[:-1],
        mu_b[:-1],
        cov_b[:-1, :-1]
    )

    # optimal weights for basket+asset at this target
    w_opt = mean_variance_opt(
        mu=mu_b,
        cov=cov_b,
        target_return=target
    )

    # if solver fails or returns None/NaNs, treat as no improvement
    if w_opt is None or not np.all(np.isfinite(w_opt)):
        return 0

    new = sharpe_ratio(w_opt, mu_b, cov_b)
    return int(new > base + tol)



# --- build dataset (fast) ---
import numpy as np
from sklearn.model_selection import train_test_split

# silence solver logs (if using cvxopt)
try:
    import cvxopt
    cvxopt.solvers.options['show_progress'] = False
except Exception:
    pass

TARGETS = [0.08, 0.13, 0.18, 0.20, 0.25]

# common precomputations
mu_vec = μ.values
N      = len(mu_vec)
Sigma  = Σ.values + 1e-6*np.eye(N)                 # ridge once for stability
idx_of = {t: i for i, t in enumerate(μ.index)}     # ticker -> index

# 1) global optimum per target (solve QP once per target)
w_star_by = {tr: mean_variance_opt(mu=mu_vec, cov=Sigma, target_return=tr)
             for tr in TARGETS}

# 2) perturbed Sharpe/CVaR per (target, candidate) (compute once, reuse)
eps = 0.01
pert_by = {}  # (tr, ticker) -> (pert_sh, pert_cv)
for tr, w_star in w_star_by.items():
    donor  = int(np.argmax(w_star))
    donor2 = int(np.argsort(w_star)[-2]) if N >= 2 else donor
    for cand in fund.index:
        j = idx_of[cand]
        d = donor2 if j == donor else donor
        w = w_star.copy()
        w[d] = max(0.0, w[d] - eps)
        w[j] += eps
        pert_by[(tr, cand)] = (
            sharpe_ratio(w, mu_vec, Sigma),
            cvar_calc(w, mu_vec, Sigma),
        )

# 3) build records using the precomputed pieces
records = []
for port in simulate_portfolios():  
    held          = set(port)
    base_tickers  = list(port.keys())
    base_w        = np.array(list(port.values()))

    mu_b  = μ[base_tickers].values
    cov_b = Σ.loc[base_tickers, base_tickers].values + 1e-6*np.eye(len(base_tickers))

    base_sh = sharpe_ratio(base_w, mu_b, cov_b)
    base_cv = cvar_calc(base_w, mu_b, cov_b)
    base_ret = float(np.dot(base_w, μ[base_tickers].values))
    base_var = float(base_w @ Σ.loc[base_tickers, base_tickers].values @ base_w)
    base_vol = float(np.sqrt(max(base_var, 1e-12)))
    
    for tr in TARGETS:
        for cand in fund.index:
            if cand in held:
                continue
            pert_sh, pert_cv = pert_by[(tr, cand)]
            cov_vec  = Σ.loc[base_tickers, cand].values          # covariances with current holdings
            cand_var = float(Σ.loc[cand, cand])
            cand_vol = float(np.sqrt(max(cand_var, 1e-12)))
            portCandCorr = float((base_w @ cov_vec) / ((base_vol + 1e-12) * (cand_vol + 1e-12)))
            
            records.append({
                "ticker":        cand,
                "targetReturn":  tr,
                "deltaSharpe":   pert_sh - base_sh,
                "deltaCvar":     base_cv - pert_cv,
                "mom6":          mom_6[cand],
                "mom12":         mom_12[cand],
                **fund.loc[cand].to_dict(),
                "targetGap":     float(tr - base_ret),
                "portCandCorr":  portCandCorr,
                "label":         label_asset(port, cand, tr), 
            })

df = pd.DataFrame(records).dropna()

feature_cols = [
    "deltaSharpe","deltaCvar",
    "mom6","mom12","beta","divYield","logCap",
    "targetGap","portCandCorr",
    "targetReturn",
]
X = df[feature_cols].astype(float)

y = df["label"]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

'''
records = []
TARGETS = [0.08, 0.13, 0.18, 0.20, 0.25]  # 8%, 13%, 18%, 20%, 25%

for port in simulate_portfolios():
    held = set(port)
    base_tickers = list(port.keys())
    base_w = np.array(list(port.values()))

    # compute base Sharpe/CVaR once per portfolio
    base_sh = sharpe_ratio(
        base_w,
        μ[base_tickers].values,
        Σ.loc[base_tickers, base_tickers].values
    )
    base_cv = cvar_calc(
        base_w,
        μ[base_tickers].values,
        Σ.loc[base_tickers, base_tickers].values
    )

    for target in TARGETS:
        # compute the global optimum for this target
        # note: mean_variance_opt solves for w* s.t. sum(w*)=1, w*>=0, μ·w*>=target
        w_star = mean_variance_opt(
            mu=μ.values,
            cov=Σ.values,
            target_return=target
        )

        # map w_star back to ticker list
        # assume UNIVERSAL ordering same as μ.index
        # so w_star[i] corresponds to μ.index[i]
        # heldIdx = [i for i,sym in enumerate(μ.index) if sym in base_tickers]

        for cand in fund.index:
            if cand in held:
                continue

            # marginal tilt of ε=1%
            eps = 0.01
            # take eps from the largest weight in w_star
            donor = np.argmax(w_star)
            w_pert = w_star.copy()
            w_pert[donor] = max(0, w_pert[donor] - eps)
            idx = list(μ.index).index(cand)
            w_pert[idx] += eps

            # compute perturbed metrics
            pert_sh = sharpe_ratio(
                w_pert,
                μ.values,
                Σ.values
            )
            pert_cv = cvar_calc(
                w_pert,
                μ.values,
                Σ.values
            )

            rec = {
                "ticker":      cand,
                "targetReturn": target,
                "deltaSharpe":  pert_sh - base_sh,
                "deltaCvar":    base_cv - pert_cv,
                "mom6":         mom_6[cand],
                "mom12":        mom_12[cand],
                **fund.loc[cand].to_dict(),
                "label":        label_asset(port, cand, target)
            }
            records.append(rec)

df = pd.DataFrame(records).dropna()
# include 'targetReturn' among features
feature_cols = [
    "deltaSharpe","deltaCvar","mom6","mom12",
    "beta","divYield","logCap","targetReturn"
]
X = df[feature_cols]
y = df["label"]

# train/val/test split as before
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)
'''

'\nrecords = []\nTARGETS = [0.08, 0.13, 0.18, 0.20, 0.25]  # 8%, 13%, 18%, 20%, 25%\n\nfor port in simulate_portfolios():\n    held = set(port)\n    base_tickers = list(port.keys())\n    base_w = np.array(list(port.values()))\n\n    # compute base Sharpe/CVaR once per portfolio\n    base_sh = sharpe_ratio(\n        base_w,\n        μ[base_tickers].values,\n        Σ.loc[base_tickers, base_tickers].values\n    )\n    base_cv = cvar_calc(\n        base_w,\n        μ[base_tickers].values,\n        Σ.loc[base_tickers, base_tickers].values\n    )\n\n    for target in TARGETS:\n        # compute the global optimum for this target\n        # note: mean_variance_opt solves for w* s.t. sum(w*)=1, w*>=0, μ·w*>=target\n        w_star = mean_variance_opt(\n            mu=μ.values,\n            cov=Σ.values,\n            target_return=target\n        )\n\n        # map w_star back to ticker list\n        # assume UNIVERSAL ordering same as μ.index\n        # so w_star[i] corresponds to μ.index[i]\n

In [58]:
df

,ticker,targetReturn,deltaSharpe,deltaCvar,mom6,mom12,beta,divYield,logCap,sector,targetGap,portCandCorr,label
0,GOOG,0.08,-0.260819,0.162698,0.164905,0.215069,1.014,0.004037,28.524157,TECHNOLOGY,-0.114288,0.363290,0
1,XOM,0.08,-0.268852,0.162267,-0.018863,-0.073851,0.502,0.036254,26.850044,ENERGY & TRANSPORTATION,-0.114288,0.377236,0
2,GOOG,0.13,-0.048936,0.118054,0.164905,0.215069,1.014,0.004037,28.524157,TECHNOLOGY,-0.064288,0.363290,0
3,XOM,0.13,-0.054197,0.117587,-0.018863,-0.073851,0.502,0.036254,26.850044,ENERGY & TRANSPORTATION,-0.064288,0.377236,0
4,GOOG,0.18,0.056957,0.066805,0.164905,0.215069,1.014,0.004037,28.524157,TECHNOLOGY,-0.014288,0.363290,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32345,CVX,0.20,0.103361,-0.036113,-0.019354,0.051369,0.841,0.043459,26.447676,ENERGY & TRANSPORTATION,0.048818,0.444619,1
32346,GOOG,0.25,0.092662,-0.138437,0.164905,0.215069,1.014,0.004037,28.524157,TECHNOLOGY,0.098818,0.364500,1
32347,JPM,0.25,0.093310,-0.138656,0.101341,0.296575,1.107,0.018184,27.406930,FINANCE,0.098818,0.598750,1
32348,XOM,0.25,0.089919,-0.139092,-0.018863,-0.073851,0.502,0.036254,26.850044,ENERGY & TRANSPORTATION,0.098818,0.395822,1


## Run Grid Search with Early Stopping

In [61]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.metrics import roc_auc_score

rng = np.random.RandomState(42)

# handle class imbalance
pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()

param_grid = {
    "max_depth":        [3, 4],
    "learning_rate":    [0.05, 0.1],
    "n_estimators":     [600],   # will be cut by early stopping
    "subsample":        [0.9],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1, 3],
    "gamma":            [0]
}

best_auc, best_params, best_model = -1, None, None

for params in ParameterGrid(param_grid):
    clf = xgb.XGBClassifier(
        objective="binary:logistic",
        tree_method="hist",
        eval_metric="auc",
        random_state=42,
        scale_pos_weight=float(pos_weight),
        early_stopping_rounds=50,
        **params
    )

    clf.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False,
    )
    best_iter = getattr(clf, "best_iteration", None)
    if best_iter is not None:
        y_val_pred = clf.predict_proba(X_val, iteration_range=(0, best_iter + 1))[:, 1]
    else:
        # fallback if attribute not present
        y_val_pred = clf.predict_proba(X_val)[:, 1]

    auc = roc_auc_score(y_val, y_val_pred)

    if auc > best_auc:
        best_auc, best_params, best_model = auc, params, clf
        print(f"New best AUC {auc:.3f} with {params} (best_iter={best_iter})")
    '''
    auc = roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1])
    if auc > best_auc:
        best_auc, best_params, best_model = auc, params, clf
        print(f"New best AUC {auc:.3f} with {params} (ntree_limit={clf.best_ntree_limit})")

print("Best validation AUC:", best_auc)
best_params
    '''

New best AUC 0.964 with {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 600, 'subsample': 0.9} (best_iter=598)
New best AUC 0.966 with {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 600, 'subsample': 0.9} (best_iter=599)
New best AUC 0.966 with {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 600, 'subsample': 0.9} (best_iter=599)
New best AUC 0.967 with {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 600, 'subsample': 0.9} (best_iter=592)


In [62]:
best_params

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 1,
 'n_estimators': 600,
 'subsample': 0.9}

## Evaluate on Test Set 

In [63]:
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report
from sklearn.metrics import roc_curve, precision_recall_fscore_support, confusion_matrix

proba_test = best_model.predict_proba(X_test)[:,1]
auc_test   = roc_auc_score(y_test, proba_test)
ap_test    = average_precision_score(y_test, proba_test)
print(f"Test AUC: {auc_test:.3f} | PR-AUC: {ap_test:.3f}")


# pick threshold on VALIDATION by maximizing Youden J
proba_val = best_model.predict_proba(X_val)[:, 1]
fpr, tpr, thr_grid = roc_curve(y_val, proba_val)
j_scores = tpr - fpr
thr = thr_grid[j_scores.argmax()]  # best threshold on validation

# apply to TEST
proba_test = best_model.predict_proba(X_test)[:, 1]
pred_test = (proba_test >= thr).astype(int)

# metrics
auc_test = roc_auc_score(y_test, proba_test)
ap_test  = average_precision_score(y_test, proba_test)
p, r, f1, _ = precision_recall_fscore_support(y_test, pred_test, average="binary", zero_division=0)
cm = confusion_matrix(y_test, pred_test)

print(f"Test AUC: {auc_test:.3f} | PR-AUC: {ap_test:.3f}")
print(f"Thr (from val): {thr:.3f} | Precision: {p:.3f}  Recall: {r:.3f}  F1: {f1:.3f}")
print("Confusion matrix [tn fp; fn tp]:\n", cm)

Test AUC: 0.968 | PR-AUC: 0.988
Test AUC: 0.968 | PR-AUC: 0.988
Thr (from val): 0.640 | Precision: 0.973  Recall: 0.854  F1: 0.910
Confusion matrix [tn fp; fn tp]:
 [[1215   85]
 [ 518 3035]]


## Save the Model

In [72]:
# --- Save the model (package-agnostic, robust) ---
from pathlib import Path
import json
from onnxmltools.convert.common.data_types import FloatTensorType
from onnxmltools.convert.xgboost import convert as convert_xgboost

FEATURE_ORDER = [
    "deltaSharpe","deltaCvar","mom6","mom12","beta","divYield","logCap",
    "targetReturn","targetGap","portCandCorr",
]

booster = best_model.get_booster()  # from xgboost.XGBClassifier
initial_types = [('input', FloatTensorType([None, len(FEATURE_ORDER)]))]
n = len(FEATURE_ORDER)  # same order you use to build X at runtime
booster = best_model.get_booster()

# Remap feature names to f0..f{n-1} so onnxmltools can parse them
booster.feature_names = [f"f{i}" for i in range(n)]
booster.feature_types = ["float"] * n  # keeps it simple

onnx_model = convert_xgboost(
    booster,
    initial_types=[('input', FloatTensorType([None, n]))],
    target_opset=13,
)


out_dir = Path("../src/lib")
out_dir.mkdir(parents=True, exist_ok=True)
(out_dir / "recommend_model.onnx").write_bytes(onnx_model.SerializeToString())
(out_dir / "recommend_columns.json").write_text(json.dumps(FEATURE_ORDER))
print("Saved:", out_dir / "recommend_model.onnx")
print("Saved:", out_dir / "recommend_columns.json")

Saved: ../src/lib/recommend_model.onnx
Saved: ../src/lib/recommend_columns.json
